In [23]:
import pandas as pd

# "patient_id exam_id spot[1-14] frame_number score"
# load patients data in a dataframe from images folder
import glob
images_paths = glob.glob("images/*.png", recursive=True)
# images are named as: patientid_examid_spotnumber_framenumber_score.png
# create a dataframe with the data removing "images/"
images_df = pd.DataFrame([path[7:-4].split("_") for path in images_paths], columns=["patient_id", "exam_id", "spot", "frame_number", "score"])
images_df["score"] = images_df["score"].astype(str)
images_df["frame_number"] = images_df["frame_number"].astype(str)
images_df["spot"] = images_df["spot"].astype(str)
images_df["patient_id"] = images_df["patient_id"].astype(str)
images_df["exam_id"] = images_df["exam_id"].astype(str)
images_df.tail()


,patient_id,exam_id,spot,frame_number,score
47716,1069,1158,9,96,0
47717,1069,1158,9,97,0
47718,1069,1158,9,98,0
47719,1069,1158,9,99,0
47720,1069,1158,9,9,0


In [24]:
# PREPARING DATA SPLITTING

# get the set of patients id
patients_ids = set(images_df["patient_id"])
patients_ids = list(patients_ids)
# data will be 70% patients in train for cross valid, and 30 for test
import random
random.shuffle(patients_ids)
train_patients = patients_ids[:int(len(patients_ids)*0.7)]
test_patients = patients_ids[int(len(patients_ids)*0.7):]
print("train patients: ", train_patients, "test patients: ", test_patients)

train patients:  ['1052', '1066', '1050', '1047', '1017', '1069', '1051'] test patients:  ['1045', '1067', '1048', '1068']


In [25]:
from torch.utils.data import Dataset
class MIDataset(Dataset):
    def __init__(self, dataset: pd.DataFrame, patient_list: list):
        tmp = dataset[dataset["patient_id"].isin(patient_list)]
        # create a list with the path of the images
        self.images_paths = ["images/" + "_".join(row) + ".png" for row in tmp.values]
        # create a list with the score of the images
        self.images_scores = tmp["score"].values
    
    def __len__(self):
        return len(self.images_paths)
    
    def __getitem__(self):
        return (self.images_paths, self.images_scores)

In [26]:
train_dataset = MIDataset(images_df, train_patients)
